BATTLESHIP GAME

In [1]:
!pip install player

In [2]:
!pip install player --upgrade
!pip install "pyramid<2"

In [6]:
from random import randint
import os
import player

# Ship Class
class Ship:
    def __init__(self, size, orientation, location):
        self.size = size
        if orientation == 'horizontal' or orientation == 'vertical':
            self.orientation = orientation
        else:
            raise ValueError("Value must be 'horizontal' or 'vertical'.")

        if orientation == 'horizontal':
            if location['row'] in range(row_size):
                self.coordinates = []
                for index in range(size):
                    if location['col'] + index in range(col_size):
                        self.coordinates.append({'row': location['row'], 'col': location['col'] + index})
                    else:
                        raise IndexError("Column is out of range.")
            else:
                raise IndexError("Row is out of range.")
        elif orientation == 'vertical':
            if location['col'] in range(col_size):
                self.coordinates = []
                for index in range(size):
                    if location['row'] + index in range(row_size):
                        self.coordinates.append({'row': location['row'] + index, 'col': location['col']})
                    else:
                        raise IndexError("Row is out of range.")
            else:
                raise IndexError("Column is out of range.")

        if self.filled():
            print_board(board)
            print(" ".join(str(coords) for coords in self.coordinates))
            raise IndexError("A ship already occupies that space.")
        else:
            self.fillBoard()

    def filled(self):
        for coords in self.coordinates:
            if board[coords['row']][coords['col']] == 1:
                return True
        return False

    def fillBoard(self):
        for coords in self.coordinates:
            board[coords['row']][coords['col']] = 1

    def contains(self, location):
        for coords in self.coordinates:
            if coords == location:
                return True
        return False

    def destroyed(self):
        for coords in self.coordinates:
            if board_display[coords['row']][coords['col']] == 'O':
                return False
            elif board_display[coords['row']][coords['col']] == '*':
                raise RuntimeError("Board display inaccurate")
        return True

# Settings Variables
row_size = 10  # number of rows
col_size = 10  # number of columns
num_ships = 5
max_ship_size = 5
min_ship_size = 2
num_turns = 100

# Create lists
ship_list = []

board = [[0] * col_size for x in range(row_size)]
board_display = [["O"] * col_size for x in range(row_size)]

# Functions

def print_board(board_array):
    print("\n  " + " ".join(str(x) for x in range(1, col_size + 1)))
    for r in range(row_size):
        print(str(r + 1) + " " + " ".join(str(c) for c in board_array[r]))
    print()

def search_locations(size, orientation):
    locations = []
    if orientation != 'horizontal' and orientation != 'vertical':
        raise ValueError("Orientation must have a value of either 'horizontal' or 'vertical'.")

    if orientation == 'horizontal':
        if size <= col_size:
            for r in range(row_size):
                for c in range(col_size - size + 1):
                    if 1 not in board[r][c:c + size]:
                        locations.append({'row': r, 'col': c})
    elif orientation == 'vertical':
        if size <= row_size:
            for c in range(col_size):
                for r in range(row_size - size + 1):
                    if 1 not in [board[i][c] for i in range(r, r + size)]:
                        locations.append({'row': r, 'col': c})

    if not locations:
        return 'None'
    else:
        return locations

def random_location(size):
    #size = randint(min_ship_size, max_ship_size)
    orientation = 'horizontal' if randint(0, 1) == 0 else 'vertical'

    locations = search_locations(size, orientation)
    if locations == 'None':
        return 'None'
    else:
        return {'location': locations[randint(0, len(locations) - 1)], 'size': size, \
                'orientation': orientation}


from random import randint
import os
#import player # removed player module import as its not needed

# ... (Rest of the existing code) ...

def get_row():
    while True:
        try:
            guess = int(input("Row Guess: "))  # Reverted to user input
            if guess in range(1, row_size + 1):
                return guess - 1
            else:
                print("\nOops, that's not even in the ocean.")
        except ValueError:
            print("\nPlease enter a number")


def get_col():
    while True:
        try:
            guess = int(input("Column Guess: "))  # Reverted to user input
            if guess in range(1, col_size + 1):
                return guess - 1
            else:
                print("\nOops, that's not even in the ocean.")
                continue
        except ValueError:
            print("\nPlease enter a number")

def get_cell():
    while True:
        try:
            row_guess = int(input("Row Guess: "))  # Reverted to user input for row
            col_guess = int(input("Column Guess: ")) # Reverted to user input for column

            if row_guess in range(1, row_size + 1) and col_guess in range(1, col_size + 1):
                return [row_guess - 1, col_guess - 1]
            else:
                print("\nOops, that's not even in the ocean.")
        except ValueError:
            print("\nPlease enter a number")

# Create the ships
temp = 0
ship_sizes = [5, 4, 3, 3, 2]
while temp < num_ships:
    ship_info = random_location(ship_sizes[temp])
    if ship_info == 'None':
        continue
    else:
        ship_list.append(Ship(ship_info['size'], ship_info['orientation'], ship_info['location']))
        temp += 1
del temp

# Play Game
os.system('cls')
print_board(board_display)

for turn in range(num_turns):
    print("Turn:", turn + 1, "of", num_turns)
    print("Ships left:", len(ship_list))
    print()

    guess_coords = {}
    while True:
        #guess_coords['row'] = get_row()
        #guess_coords['col'] = get_col()
        guess_coords['row'] = get_cell()[0]
        guess_coords['col'] = get_cell()[1]

        if board_display[guess_coords['row']][guess_coords['col']] == 'X' or \
                board_display[guess_coords['row']][guess_coords['col']] == '*':
            print("You guessed that one already.")
            print("Value of the cell is: ", board_display[guess_coords['row']][guess_coords['col']])
            continue
        else:
            break

    os.system('cls')
    ship_hit = False
    for ship in ship_list:
        if ship.contains(guess_coords):
            print("Hit!")
            ship_hit = True
            board_display[guess_coords['row']][guess_coords['col']] = 'X'
            if ship.destroyed():
                print("Ship Destroyed!")
                ship_list.remove(ship)
            break
    if not ship_hit:
        board_display[guess_coords['row']][guess_coords['col']] = '*'
        print("You missed!")

    print_board(board_display)

    if not ship_list:
        break

# End Game
if ship_list:
    print("You lose!")
else:
    print("All the ships are sunk. You win!")
print("You took ", turn+1, " turns to win")


  1 2 3 4 5 6 7 8 9 10
1 O O O O O O O O O O
2 O O O O O O O O O O
3 O O O O O O O O O O
4 O O O O O O O O O O
5 O O O O O O O O O O
6 O O O O O O O O O O
7 O O O O O O O O O O
8 O O O O O O O O O O
9 O O O O O O O O O O
10 O O O O O O O O O O

Turn: 1 of 100
Ships left: 5

Row Guess: 7
Column Guess: 5
Row Guess: 3
Column Guess: 2
You missed!

  1 2 3 4 5 6 7 8 9 10
1 O O O O O O O O O O
2 O O O O O O O O O O
3 O O O O O O O O O O
4 O O O O O O O O O O
5 O O O O O O O O O O
6 O O O O O O O O O O
7 O * O O O O O O O O
8 O O O O O O O O O O
9 O O O O O O O O O O
10 O O O O O O O O O O

Turn: 2 of 100
Ships left: 5

Row Guess: 5
Column Guess: 4
Row Guess: 3
Column Guess: 5
You missed!

  1 2 3 4 5 6 7 8 9 10
1 O O O O O O O O O O
2 O O O O O O O O O O
3 O O O O O O O O O O
4 O O O O O O O O O O
5 O O O O * O O O O O
6 O O O O O O O O O O
7 O * O O O O O O O O
8 O O O O O O O O O O
9 O O O O O O O O O O
10 O O O O O O O O O O

Turn: 3 of 100
Ships left: 5

Row Guess: 7
Column Guess: 7
Row